In [4]:
import requests
from bs4 import BeautifulSoup
import json
import re

###--- 流行語大賞のワードをユーキャンのホームページから取得して、jsonに保存する
def makeJson (url):
    # URL of the webpage to scrape
    # url = "https://www.jiyu.co.jp/singo/index.php?eid=00001";

    # Send a GET request to the webpage
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the webpage
        soup = BeautifulSoup(response.content, "html.parser")
        
        year = soup.find('h2', class_='h2').text
        
        awardentrys = soup.find_all('div', class_='awardentry')
        awards = {"year":year, "awards":[]}
        
        for awardentry in awardentrys:
            award = {}
            award["award"] = awardentry.select('div.label')[0].text
            award["ward"] = awardentry.select('h3')[0].text
            award["winner"] = awardentry.select('span.winner')[0].text
            award["text"] = str(awardentry.select('p.p')[0].text).replace("\n", "");
            awards["awards"].append(award)

        nominnees = soup.find_all('ul', class_='nominee')
        if (len(nominnees) > 0):
            nominnees = nominnees[0].find_all('li')
            awards["nominees"] = []
            for li in nominnees:
                awards["nominees"].append(li.select('em')[0].text)
        
        print(awards)
        with open('data/' + re.search(r'\d{4}', year)[0] + '.json', 'w', encoding='UTF-8') as file: # 第二引数：writableオプションを指定
            json.dump(awards, file, indent=2, ensure_ascii=False)
    else:
        print("Failed to retrieve the webpage")

    response.close()
    

###--- main

url_ori = 'https://www.jiyu.co.jp/singo/index.php?eid=000';

for i in range(1, 41):
    url = url_ori + str(i).zfill(2)
    makeJson(url=url)

# makeJson(url='https://www.jiyu.co.jp/singo/index.php?eid=00040')

IndexError: list index out of range